Detección de caras con webcam

In [1]:
import cv2
import dlib
import numpy as np

En el bucle de procesamiento, la tecla 'd' permite cambiar entre filtros

La ejecución de la siguiente celda produce error al no disponer de los archivos shape_predictor_5_face_landmarks.dat y shape_predictor_68_face_landmarks.dat que por su tamaño no se incluyeron en el repositorio. Pueden descargarse desde el enlace proporcionado en el campus virtual (opción aconsejada), o
desde el [repositorio de archivos de dlib](http://dlib.net/files/).



In [4]:
# Inicializar el detector de caras y puntos faciales
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Cargar las imágenes de la peluca de Goku en estado normal y en Super Saiyan con canal alfa
goku_base = cv2.imread("goku_base1.png", -1)
goku_saiyan = cv2.imread("goku_saiyan.png", -1)

def align_image_to_face(peluca, point1, point2, point3, face_point1, face_point2, face_point3):
    # Definir tres puntos de referencia en la imagen de la peluca
    peluca_points = np.float32([point1, point2, point3])
    face_points = np.float32([face_point1, face_point2, face_point3])

    # Calcular la transformación afín
    transformation_matrix = cv2.getAffineTransform(peluca_points, face_points)

    # Aplicar la transformación afín al tamaño del `frame` para cubrir toda la cara
    aligned_peluca = cv2.warpAffine(peluca, transformation_matrix, (frame.shape[1], frame.shape[0]), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

    return aligned_peluca

def is_mouth_open(landmarks, threshold=15):
    # Calcular la distancia entre el labio superior (punto 62) y el labio inferior (punto 66)
    top_lip = landmarks.part(62).y
    bottom_lip = landmarks.part(66).y
    mouth_open_distance = abs(bottom_lip - top_lip)
    
    # Verificar si la distancia supera el umbral para determinar si la boca está abierta
    return mouth_open_distance > threshold


filter = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detectar caras en el marco
    faces = detector(frame)

    for face in faces:
        # Obtener landmarks faciales
        landmarks = predictor(frame, face)

        if filter==0:
            # Determinar si la boca está abierta
            if is_mouth_open(landmarks):
                peluca_actual = goku_saiyan  # Cambiar a la peluca de Goku Super Saiyan si la boca está abierta
            else:
                peluca_actual = goku_base  # Usar la peluca base si la boca está cerrada

            # Obtener tres puntos de referencia en la cara
            forehead_left = (landmarks.part(0).x, landmarks.part(0).y)    # Esquina izquierda de la frente
            forehead_right = (landmarks.part(16).x, landmarks.part(16).y) # Esquina derecha de la frente
            chin = (landmarks.part(8).x, landmarks.part(8).y)             # Punto de la barbilla

            # Definir tres puntos en la peluca para alinear
            peluca_left = (150, 300)                    # Punto en el borde izquierdo de la peluca
            peluca_right = (350, 300)                   # Punto en el borde derecho de la peluca
            peluca_chin = (250, 400)                    # Punto en la parte inferior de la peluca (centro)

            # Alinear la peluca con los puntos de la cara
            aligned_peluca = align_image_to_face(peluca_actual, peluca_left, peluca_right, peluca_chin, forehead_left, forehead_right, chin)

            # Componer la peluca en el marco principal
            alpha_peluca = aligned_peluca[:, :, 3] / 255.0
            for c in range(0, 3):
                frame[:, :, c] = (alpha_peluca * aligned_peluca[:, :, c] + (1 - alpha_peluca) * frame[:, :, c])
        if filter == 1:
            #implementar segundo filtro
            frame=frame

    # Mostrar el marco con la peluca aplicada
    cv2.imshow('Filtros', frame)
    key = cv2.waitKey(1)
    if key == 27:  # Presiona Esc para salir
        break

    elif key == ord('d'):  # Presiona "d" para alternar entre los modos 
        if filter == 0:
            filter = 1
        else:
            filter = 0

cap.release()
cv2.destroyAllWindows()
